# Applying our methods to CERN ghost track data

### Imports

In [ ]:
#%pip install matplotlib numpy qiskit scipy pandas qiskit_machine_learning natsort tqdm

In [101]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import BasicAer
from qiskit import Aer, transpile

from qiskit.utils import QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZFeatureMap
from qiskit.circuit.library import StatePreparation

from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.kernels import FidelityQuantumKernel

import qiskit_machine_learning.kernels
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data


from sklearn.svm import SVC, OneClassSVM
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split
import random
from sklearn.datasets import make_blobs

import scipy.io
import pandas as pd
from qiskit.providers.aer import AerError
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit.algorithms.state_fidelities import ComputeUncompute
algorithm_globals.random_seed = 0
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from qiskit_machine_learning.circuit.library import RawFeatureVector

from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.quantum_info import DensityMatrix
from functools import reduce
from sympy import Matrix
from sympy import sqrt as special_sqrt
from qiskit import *
from qiskit.extensions import HamiltonianGate
from qiskit.quantum_info import Operator

import os
from natsort import natsorted
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


## Loading data

Label 1 represents the inlier class (regular tracks), label -1 the outlier class (ghost tracks).

In [28]:
# load all json files in cern_data folder
cern_data = []
for file in natsorted(os.listdir('cern_data')):
    if file.endswith('.json'):
        df = pd.read_json('cern_data/' + file)
        cern_data.append(df)

cern_data = pd.concat(cern_data, ignore_index=True)
# replace 1 (ghost) with -1 (outlier) and 0 (regular) with 1 (inlier)
cern_data['is_ghost'] = cern_data['is_ghost'].replace(1, -1).replace(0, 1)
cern_data

,chi2,is_ghost,kalman_ip_chi2,nb_hits,ndof,qop,tx,ty,x,y,z
0,37.042889,1,2.744781,29,22,0.000093,-0.032356,-0.026533,0.032351,-0.039451,2.051071
1,12.333838,1,505.177429,21,6,-0.000100,-0.020835,-0.022025,-1.026810,0.971350,124.333954
2,11.791684,1,1.248908,24,12,0.000113,-0.008289,-0.035582,-0.032666,0.007609,84.878311
3,22.339798,1,3893.681641,26,16,-0.000053,-0.004296,-0.019615,0.003717,-0.000814,-109.656128
4,19.041876,1,2.024810,25,14,0.000094,-0.007948,-0.046065,0.069844,-0.012051,84.299057
...,...,...,...,...,...,...,...,...,...,...,...
18320,21.877516,1,1.353087,31,26,-0.000246,-0.003892,0.094137,-0.010576,-0.000436,-133.640045
18321,17.455692,1,0.945140,22,8,-0.000223,-0.170432,-0.096830,0.027389,-0.048208,94.859215
18322,2.377720,1,1.130440,20,6,-0.000157,-0.092891,-0.061437,0.007144,-0.010802,94.624451
18323,7.393714,1,10.609664,26,18,-0.000206,0.122870,-0.017398,0.016633,0.117464,-32.473801


In [29]:
cern_data['is_ghost'].value_counts()

 1    17705
-1      620
Name: is_ghost, dtype: int64

In [34]:
cern_data[cern_data['is_ghost'] == -1]

,chi2,is_ghost,kalman_ip_chi2,nb_hits,ndof,qop,tx,ty,x,y,z
114,6.736708,-1,2.013437,17,4,-0.000317,0.156288,-0.067397,0.011041,0.025599,196.992310
122,10.666078,-1,0.258542,21,8,-0.000028,-0.020742,-0.000281,-0.000607,0.044702,9.808350
123,91.954224,-1,41.465874,21,8,-0.000029,-0.020258,-0.000281,-0.000660,0.047713,-1.533783
124,7.582039,-1,1.203741,21,8,-0.000029,-0.019606,-0.000397,-0.000422,0.020831,9.954773
146,23.850941,-1,0.210660,29,24,0.000157,-0.034494,0.020627,-0.011212,-0.018750,-30.769402
...,...,...,...,...,...,...,...,...,...,...,...
18081,14.100121,-1,6.466623,26,18,-0.000182,0.086158,-0.034164,0.022217,0.056031,-27.052513
18150,210.464294,-1,7.795339,24,14,-0.000118,0.018131,0.093354,-0.075147,0.014596,-13.119804
18261,4.791725,-1,4.774308,17,4,0.000014,-0.001622,0.015018,-0.071725,-0.007744,109.908081
18263,15.251548,-1,0.920522,24,12,0.000086,-0.016061,0.033375,-0.023291,-0.011209,116.459412


### Preprocessing

Standardize features (subtract mean and scale to unit variance)

In [85]:
std_cern_data = StandardScaler().fit_transform(cern_data.drop(columns=['is_ghost']))
std_cern_data = pd.DataFrame(std_cern_data, columns=cern_data.drop(columns=['is_ghost']).columns)
std_cern_data['is_ghost'] = cern_data['is_ghost']
std_cern_data

,chi2,kalman_ip_chi2,nb_hits,ndof,qop,tx,ty,x,y,z,is_ghost
0,0.149075,-0.190888,1.238320,1.094042,0.619355,-0.510660,-0.378255,0.038910,-0.057279,-0.083012,1
1,-0.116673,0.143397,-1.279191,-1.529440,-0.652187,-0.331165,-0.312240,-1.086172,1.078666,1.402944,1
2,-0.122504,-0.191883,-0.335124,-0.545634,0.745963,-0.135685,-0.510762,-0.030154,-0.004392,0.923487,1
3,-0.009058,2.397880,0.294253,0.110236,-0.343631,-0.073471,-0.276957,0.008494,-0.013859,-1.440454,1
4,-0.044528,-0.191367,-0.020435,-0.217699,0.624156,-0.130381,-0.664273,0.078736,-0.026486,0.916448,1
...,...,...,...,...,...,...,...,...,...,...,...
18320,-0.014030,-0.191813,1.867697,1.749913,-1.610304,-0.067185,1.388742,-0.006689,-0.013434,-1.731901,1
18321,-0.061587,-0.192085,-0.964502,-1.201505,-1.455961,-2.661969,-1.407636,0.033639,-0.067121,1.044773,1
18322,-0.223752,-0.191962,-1.593879,-1.529440,-1.026368,-1.453845,-0.889370,0.012134,-0.025083,1.041920,1
18323,-0.169805,-0.185655,0.294253,0.438172,-1.343648,1.907848,-0.244495,0.022214,0.119063,-0.502550,1


## Custom Feature Maps

In [30]:
def custom_data_map_func(x):
    mapped = x[0] if len(x) == 1 else reduce(lambda m, n: m * n, x)
    return mapped
def feature_map_superfidel(x):
    # as described in 
    # https://doi.org/10.1103/PhysRevA.97.042315
    
    # Qiskit currently doesn't natively support a square root function in a parameter expression
    # So use sympy base to get the same effect
    mapped = x[0] if len(x) == 1 else reduce(lambda m, n: m * n, 
                                             np.divide(x,(1-np.square(np.column_stack(x)).trace())._call(special_sqrt)))
    return mapped

## Quantum Function for OneClass (Algorithm 1)

In [65]:
# First algorithm, returns trained model
def Algorithm1(X, y, reps=2, shots=1, outliers_fraction=20/210,
               entanglement="linear", num_features = 2, seed = 0, 
               supervised=False, feature_map_no = 1, data_map_no = 1,
               svm_max_iter=-1) :
    if feature_map_no == 1:
        # Define ZZFeatureMap using inputs
        if data_map_no == 1:
             feature_map = ZZFeatureMap(feature_dimension = num_features, 
                                   reps = reps, entanglement=entanglement)
        elif data_map_no == 2:
            feature_map = ZZFeatureMap(feature_dimension = num_features, 
                                   reps = reps, entanglement=entanglement, 
                                       data_map_func = custom_data_map_func)
        elif data_map_no == 3:
            feature_map = ZZFeatureMap(feature_dimension = num_features, 
                                   reps = reps, entanglement=entanglement, 
                                       data_map_func = feature_map_superfidel)
    elif feature_map_no == 2:
        # Define ZZFeatureMap using inputs
        feature_map = ZFeatureMap(feature_dimension = num_features, reps = reps)
    elif feature_map_no == 3:
        # Define ZZFeatureMap using inputs
        feature_map = PauliFeatureMap(feature_dimension = num_features, reps = reps)
    # Calculates probabilities of bit results from quantum circuits
    sampler = Sampler()
    # uses sampler to calculate state fidelity of 2 quantum circuits
    fidelity = ComputeUncompute(sampler=sampler)
    # Translates data with base state fidelity distance metric
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    # Kernel needs to be evaluated before going into the One-Class SVM
    svm = OneClassSVM(kernel = kernel.evaluate, verbose=True, nu=outliers_fraction, max_iter=svm_max_iter)
    if supervised: 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=seed)
        svm.fit(X_train, y_train)
        y_pred = svm.predict(X_test)
        # TODO save to Matrix
#         print(classification_report(y_test, y_pred))
        print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

        print("Precision 1: {}".format(precision_score(y_test, y_pred, average='binary')))
        print("Precision -1: {}".format(precision_score(y_test, y_pred, pos_label=-1, average='binary')))

        print("Recall 1: {}".format(recall_score(y, y_pred, average='macro')))
        print("F1 1: {}".format(f1_score(y, y_pred, average='macro')))
#         print("Recall 1: {}".format(recall_score(y_test, y_pred, average='binary')))
#         print("Recall -1: {}".format(recall_score(y_test, y_pred, pos_label=-1, average='binary')))

#         print("F1 1: {}".format(f1_score(y_test, y_pred, average='binary')))
#         print("F1 -1: {}".format(f1_score(y_test, y_pred, pos_label=-1, average='binary')))
    else: 
        svm.fit(X)
        y_pred = svm.predict(X)
        #TODO save to matrix
#         print(classification_report(y, y_pred))
        print("Accuracy: {}".format(accuracy_score(y, y_pred)))

        print("Precision 1: {}".format(precision_score(y, y_pred, average='binary')))
        print("Precision -1: {}".format(precision_score(y, y_pred, pos_label=-1, average='binary')))
        
        print("Macro Recall: {}".format(recall_score(y, y_pred, average='macro')))
        print("Macro F1: {}".format(f1_score(y, y_pred, average='macro')))

#         print("Recall 1: {}".format(recall_score(y, y_pred, average='binary')))
#         print("Recall -1: {}".format(recall_score(y, y_pred, pos_label=-1, average='binary')))

#         print("F1 1: {}".format(f1_score(y, y_pred, average='binary')))
#         print("F1 -1: {}".format(f1_score(y, y_pred, pos_label=-1, average='binary')))

    return svm

## Preparing data

In [96]:
num_samples = 300

cern_sample = std_cern_data.sample(num_samples, random_state=0)

# select input features
X = cern_sample[['x', 'y', 'z']] 
y = cern_sample['is_ghost']

print(y.value_counts())

 1    286
-1     14
Name: is_ghost, dtype: int64


## Applying Algorithm 1 to data

In [97]:
trained_svm = Algorithm1(X, y, reps=1, num_features=3, svm_max_iter=-1)

[LibSVM]*.*
optimization finished, #iter = 493
obj = 55.918164, rho = 4.101344
nSV = 43, nBSV = 19
Accuracy: 0.8666666666666667
Precision 1: 0.9589552238805971
Precision -1: 0.09375
Macro Recall: 0.5564435564435565
Macro F1: 0.5291163082718568


**Runtimes:**

*num_samples, svm_max_iter, runtime, macroF1*  
100, 2, 13s  
100, 10, 21s  
100, 258(-1), 16s, 0.46  

200, 1, 1m3s, 0.24  
200, 240(-1), 1m3s, 0.57  

300, 493(-1), 2m28s


In [111]:
cern_test_sample = std_cern_data.sample(100, random_state=1)

# select input features
X = cern_test_sample[['x', 'y', 'z']] 
y = cern_test_sample['is_ghost']

print(y.value_counts())

y_pred = trained_svm.predict(X)

print("Macro F1: {}".format(f1_score(y, y_pred, average='macro')))

#for row in tqdm(range(100)):
#    X = X_full.iloc[-row] 
#    y_pred = trained_svm.predict(X)


 1    97
-1     3
Name: is_ghost, dtype: int64


Inference runtime is long! TODO Remove from Algorithm1 function  
100, 50s, 0.47F1  

## Comparison to classical one-class SVM methods

In [68]:
other_kernel_list = ['rbf', 'linear', 'poly', 'sigmoid']
outliers_fraction=20/210
supervised = False

print()
for kernel in other_kernel_list:
    svm_classical = OneClassSVM(kernel = kernel, verbose=True,  nu=outliers_fraction)
    if supervised:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
        svm_classical.fit(X_train, y_train)
        y_pred = svm_classical.predict(X_test)
        # TODO save to Matrix
        print("{}: ".format(kernel))
#             print(classification_report(y_test, y_pred))
        print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))
        
        print("Precision 1: {}".format(precision_score(y_test, y_pred, average='binary')))
        print("Precision -1: {}".format(precision_score(y_test, y_pred, pos_label=-1, average='binary')))
        
        print("Recall 1: {}".format(recall_score(y, y_pred, average='macro')))
        print("F1 1: {}".format(f1_score(y, y_pred, average='macro')))
        
#             print("Recall 1: {}".format(recall_score(y_test, y_pred, average='binary')))
#             print("Recall -1: {}".format(recall_score(y_test, y_pred, pos_label=-1, average='binary')))
        
#             print("F1 1: {}".format(f1_score(y_test, y_pred, average='binary')))
#             print("F1 -1: {}".format(f1_score(y_test, y_pred, pos_label=-1, average='binary')))

    else:
        svm_classical.fit(X)
        y_pred = svm_classical.predict(X)
        # TODO save to Matrix

        print("{}: ".format(kernel))
#             print(classification_report(y, y_pred))
        print("Accuracy: {}".format(accuracy_score(y, y_pred)))
        
        print("Precision 1: {}".format(precision_score(y, y_pred, average='binary')))
        print("Precision -1: {}".format(precision_score(y, y_pred, pos_label=-1, average='binary')))
        
        print("Recall 1: {}".format(recall_score(y, y_pred, average='macro')))
        print("F1 1: {}".format(f1_score(y, y_pred, average='macro')))
#             print("Recall 1: {}".format(recall_score(y, y_pred, average='binary')))
#             print("Recall -1: {}".format(recall_score(y, y_pred, pos_label=-1, average='binary')))
        
#             print("F1 1: {}".format(f1_score(y, y_pred, average='binary')))
#             print("F1 -1: {}".format(f1_score(y, y_pred, pos_label=-1, average='binary')))




For dataset: 0
[LibSVM]rbf: 
*
optimization finished, #iter = 379
obj = 993.095414, rho = 25.013974
nSV = 101, nBSV = 91
Accuracy: 0.864
Precision 1: 0.9630872483221476
Precision -1: 0.02830188679245283
Recall 1: 0.4882434301521439
F1 1: 0.484528267560151
[LibSVM].............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
*................................................................................................................................................................................................................................................................